In [1]:
# Project goal: Predict MLB batters average yearly salary based on production stats from 5 yrs prior to free agency
# outline:
# Clean data
# EDA on production stats vs salary info
# ideas:
# 1. explore, plot relationship between individual stats and salary
# 2. variable selection using step-wise regression
# 3. explore corralation between salary vs basic stats & advanced stats

In [2]:
import pandas as pd
from pybaseball import playerid_lookup
from pybaseball import player_search_list
import numpy as np

pd.set_option('display.max_rows', 10)

In [3]:
# clean, feature eng salary data:

salary_df = pd.read_csv('mlb-free-agency.csv')
print(len(salary_df.index))
salary_df.head()

1613


,Unnamed: 0,name,position,age,from_team,to_tam,contract_length,total_salary,avg_salary,year
0,0,Wil Nieves,C,33.2,WSH,MIL,1,775000.0,775000.0,2011
1,1,Albert Pujols,DH,31.8,STL,LAA,10,240000000.0,24000000.0,2012
2,2,Prince Fielder,DH,27.7,MIL,DET,9,214000000.0,23777778.0,2012
3,3,Jose Reyes,SS,28.4,NYM,MIA,6,106000000.0,17666667.0,2012
4,4,C.J. Wilson,SP,31.0,TEX,LAA,5,77500000.0,15500000.0,2012


In [4]:
# check how many entries have no salary info
print(len(salary_df[salary_df.isnull().any(axis=1)].index))

392


In [5]:
# drop rows without salary data
salary_df = salary_df[salary_df['total_salary'].notna()]
len(salary_df.index)

1221

In [6]:
# drop first row since only 1 observation in 2011
salary_df = salary_df.drop([0])
len(salary_df.index)

1220

In [7]:
# drop first column and reset index
salary_df.drop(columns=salary_df.columns[0], 
        axis=1, 
        inplace=True)
salary_df.reset_index(drop=True)

,name,position,age,from_team,to_tam,contract_length,total_salary,avg_salary,year
0,Albert Pujols,DH,31.8,STL,LAA,10,240000000.0,24000000.0,2012
1,Prince Fielder,DH,27.7,MIL,DET,9,214000000.0,23777778.0,2012
2,Jose Reyes,SS,28.4,NYM,MIA,6,106000000.0,17666667.0,2012
3,C.J. Wilson,SP,31.0,TEX,LAA,5,77500000.0,15500000.0,2012
4,Mark Buehrle,SP,32.7,CHW,MIA,4,58000000.0,14500000.0,2012
...,...,...,...,...,...,...,...,...,...
1215,Sam McWilliams,RP,25.2,TB,NYM,1,750000.0,750000.0,2021
1216,Kohl Stewart,SP,26.2,BAL,CHC,1,700000.0,700000.0,2021
1217,Scott Heineman,OF,27.9,TEX,TEX,1,595000.0,595000.0,2021
1218,Albert Pujols,DH,41.2,LAA,LAD,1,570500.0,570500.0,2021


In [8]:
# since salary data from spotrac.com don't come with unique playerID, I need to assign each player and fangraphID
# create a list of unique player names, and see how many there are
players = salary_df.name.unique()

players = list(players)
print(len(players))

777


In [9]:
# look up fangraphID by name
players[:10]

['Albert Pujols',
 'Prince Fielder',
 'Jose Reyes',
 'C.J. Wilson',
 'Mark Buehrle',
 'Jonathan Papelbon',
 'Aramis Ramirez',
 'Michael Cuddyer',
 'Heath Bell',
 'Carlos Beltran']

In [10]:
player_list = []
for i in players:
    name = i.split(' ')
    first_name = name[0].lower()
    last_name = name[-1].lower()
    name_tup = (last_name, first_name)
    player_list.append(name_tup)

playerid = player_search_list(player_list)
playerid

Gathering player lookup table. This may take a moment.


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,pujols,albert,405395,pujoa001,pujolal01,1177,2001.0,2021.0
1,fielder,prince,425902,fielp001,fieldpr01,4613,2005.0,2016.0
2,reyes,jose,431170,reyej002,reyesjo02,2797,2006.0,2006.0
3,reyes,jose,408314,reyej001,reyesjo01,1736,2003.0,2018.0
4,buehrle,mark,279824,buehm001,buehrma01,225,2000.0,2015.0
...,...,...,...,...,...,...,...,...
755,brebbia,john,605154,brebj001,brebbjo01,12777,2017.0,2021.0
756,middleton,keynan,641871,middk001,middlke01,15264,2017.0,2021.0
757,drake,oliver,543118,drako001,drakeol01,8823,2015.0,2020.0
758,holder,jonathan,656547,holdj002,holdejo02,16588,2016.0,2020.0


In [11]:
playerid['full_name'] = playerid['name_first'] + ' ' + playerid['name_last']

In [12]:
# check the names of plyers that didn't turn up serach result
unique_names = list(playerid['full_name'].unique())
players_missing = []
for i in players:
    if i.lower() not in unique_names:
        players_missing.append(i.lower())
players_missing

['c.j. wilson',
 'jerry hairston jr.',
 'vincent padilla',
 'melvin upton',
 'a.j. pierzynski',
 'j.p. howell',
 'juan carlos oviedo',
 'a.j. burnett',
 'j.p. arencibia',
 'yorbit torrealba',
 'j.c. gutiérrez',
 'john mayberry jr.',
 'eric young jr.',
 'nick massett',
 'jeremy mcbryde',
 'j.a. happ',
 'alejandro de aza',
 'norichika aoki',
 'steven pearce',
 'alexei ramirez',
 'alberto alburquerque',
 'justin de fratus',
 'a.j. griffin',
 'yoenis cespedes',
 'michael dunn',
 'r.a. dickey',
 'a.j. ellis',
 'tom milone',
 'j.d. martinez',
 'c.c. sabathia',
 'michael fiers',
 'seung-hwan oh',
 'leonys martin',
 'a.j. pollock',
 'd.j. lemahieu',
 'hyun-jin ryu',
 'c.j. cron',
 'jose iglesias',
 'alexander claudio',
 'c.j. edwards',
 'daniel winkler',
 'j.t. realmuto',
 'jackie bradley jr.',
 'tommy la stella',
 'daniel santana',
 'sam mcwilliams',
 'joe gatto']

In [13]:
print(f"Number of players not searched: {len(players_missing)}")

Number of players not searched: 47


In [14]:
# there are quite a few missing players and formatting and searching all of them is too tedious
# I decided to search only players with abbreviated first name, e.g: c.j. wilson

# come up with a list of tuple of player names that contain '.', and turn them into a list of tuple for ID search
# also, insert an empty space after the first '.' so the last name complies with ID search format
players_missing_tup = []
for i in players_missing:
    name = i.split(' ')
    if '.' not in name[0]:
        continue
    first_name = name[0].replace('.', '. ', 1).lower()
    last_name = name[-1].lower()
    name_tup = (last_name, first_name)
    players_missing_tup.append(name_tup)
print(f"Number of players with abbreviated name: {len(players_missing_tup)}")
players_missing_tup

Number of players with abbreviated name: 17


[('wilson', 'c. j.'),
 ('pierzynski', 'a. j.'),
 ('howell', 'j. p.'),
 ('burnett', 'a. j.'),
 ('arencibia', 'j. p.'),
 ('gutiérrez', 'j. c.'),
 ('happ', 'j. a.'),
 ('griffin', 'a. j.'),
 ('dickey', 'r. a.'),
 ('ellis', 'a. j.'),
 ('martinez', 'j. d.'),
 ('sabathia', 'c. c.'),
 ('pollock', 'a. j.'),
 ('lemahieu', 'd. j.'),
 ('cron', 'c. j.'),
 ('edwards', 'c. j.'),
 ('realmuto', 'j. t.')]

In [15]:
# do another ID search on players with abbreviated first name, store in playerid_abb

playerid_abb = player_search_list(players_missing_tup)
playerid_abb
# we don't have a ton of new search results but it's better than nothing

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,wilson,c. j.,450351,wilsc004,wilsocj01,3580,2005.0,2015.0
1,pierzynski,a. j.,150229,piera001,pierza.01,746,1998.0,2016.0
2,howell,j. p.,434442,howej003,howeljp01,8245,2005.0,2017.0
3,burnett,a. j.,150359,burna001,burnea.01,512,1999.0,2015.0
4,arencibia,j. p.,450317,arenj001,arencjp01,697,2010.0,2015.0
...,...,...,...,...,...,...,...,...
7,dickey,r. a.,285079,dickr001,dicker.01,1245,2001.0,2017.0
8,ellis,a. j.,454560,ellia001,ellisaj01,5677,2008.0,2018.0
9,martinez,j. d.,502110,martj006,martijd02,6184,2011.0,2021.0
10,cron,c. j.,543068,cronc002,croncj01,12546,2014.0,2021.0


In [16]:
# add "full_name" into the new search results, then append new results into playerid df
playerid_abb['full_name'] = playerid_abb['name_first'] + ' ' + playerid['name_last']

playerid = playerid.append(playerid_abb)
playerid.shape
# The total number of IDs will be more than # of unique player names because there are multiple players with the same name

(772, 9)

In [17]:
# since the salary data contains info starting from 2011, and we consider player stats 5 yrs prior to the salary year

# we drop players who retired before 2006

index_names = playerid[playerid['mlb_played_last'] <= 2006 ].index
playerid = playerid.drop(index_names)

playerid.shape

(750, 9)

In [18]:
# also we will drop the mlbam, key_retro, and key_bbref columns
playerid = playerid.drop(columns=['key_mlbam', 'key_retro', 'key_bbref'])
playerid

,name_last,name_first,key_fangraphs,mlb_played_first,mlb_played_last,full_name
0,pujols,albert,1177,2001.0,2021.0,albert pujols
1,fielder,prince,4613,2005.0,2016.0,prince fielder
3,reyes,jose,1736,2003.0,2018.0,jose reyes
4,buehrle,mark,225,2000.0,2015.0,mark buehrle
5,papelbon,jonathan,5975,2005.0,2016.0,jonathan papelbon
...,...,...,...,...,...,...
7,dickey,r. a.,1245,2001.0,2017.0,r. a. cuddyer
8,ellis,a. j.,5677,2008.0,2018.0,a. j. bell
9,martinez,j. d.,6184,2011.0,2021.0,j. d. beltran
10,cron,c. j.,12546,2014.0,2021.0,c. j. willingham


In [19]:
mask = playerid.duplicated(subset=['full_name'], keep=False)
playerid[mask]

,name_last,name_first,key_fangraphs,mlb_played_first,mlb_played_last,full_name
79,young,chris,3196,2004.0,2017.0,chris young
80,young,chris,3882,2006.0,2018.0,chris young
163,hernandez,roberto,3273,2006.0,2016.0,roberto hernandez
164,hernandez,roberto,605,1991.0,2007.0,roberto hernandez
272,rodriguez,francisco,1642,2002.0,2017.0,francisco rodriguez
...,...,...,...,...,...,...
561,jennings,dan,-1,NaN,NaN,dan jennings
634,smith,will,8048,2012.0,2021.0,will smith
635,smith,will,19197,2019.0,2021.0,will smith
738,taylor,michael,11489,2014.0,2021.0,michael taylor


In [20]:
# drop all players(including the origional one) if other players with same name exist
playerid = playerid.drop_duplicates(subset=['full_name'], keep=False)
# reset index
playerid = playerid.reset_index(drop=True)
playerid

,name_last,name_first,key_fangraphs,mlb_played_first,mlb_played_last,full_name
0,pujols,albert,1177,2001.0,2021.0,albert pujols
1,fielder,prince,4613,2005.0,2016.0,prince fielder
2,reyes,jose,1736,2003.0,2018.0,jose reyes
3,buehrle,mark,225,2000.0,2015.0,mark buehrle
4,papelbon,jonathan,5975,2005.0,2016.0,jonathan papelbon
...,...,...,...,...,...,...
727,dickey,r. a.,1245,2001.0,2017.0,r. a. cuddyer
728,ellis,a. j.,5677,2008.0,2018.0,a. j. bell
729,martinez,j. d.,6184,2011.0,2021.0,j. d. beltran
730,cron,c. j.,12546,2014.0,2021.0,c. j. willingham


In [21]:
# reformat full_name so that it is the same as the full name in the salary data
playerid['full_name_new'] = playerid['name_first'].str.strip() + playerid['name_last']
playerid.head()

,name_last,name_first,key_fangraphs,mlb_played_first,mlb_played_last,full_name,full_name_new
0,pujols,albert,1177,2001.0,2021.0,albert pujols,albertpujols
1,fielder,prince,4613,2005.0,2016.0,prince fielder,princefielder
2,reyes,jose,1736,2003.0,2018.0,jose reyes,josereyes
3,buehrle,mark,225,2000.0,2015.0,mark buehrle,markbuehrle
4,papelbon,jonathan,5975,2005.0,2016.0,jonathan papelbon,jonathanpapelbon


In [22]:
# add full_name_new column to salary_df for joining
salary_df['full_name_new'] = salary_df['name'].str.lower()
salary_df['full_name_new'] = salary_df['full_name_new'].str.replace(' ', '')

salary_df

,name,position,age,from_team,to_tam,contract_length,total_salary,avg_salary,year,full_name_new
1,Albert Pujols,DH,31.8,STL,LAA,10,240000000.0,24000000.0,2012,albertpujols
2,Prince Fielder,DH,27.7,MIL,DET,9,214000000.0,23777778.0,2012,princefielder
3,Jose Reyes,SS,28.4,NYM,MIA,6,106000000.0,17666667.0,2012,josereyes
4,C.J. Wilson,SP,31.0,TEX,LAA,5,77500000.0,15500000.0,2012,c.j.wilson
5,Mark Buehrle,SP,32.7,CHW,MIA,4,58000000.0,14500000.0,2012,markbuehrle
...,...,...,...,...,...,...,...,...,...,...
1597,Sam McWilliams,RP,25.2,TB,NYM,1,750000.0,750000.0,2021,sammcwilliams
1598,Kohl Stewart,SP,26.2,BAL,CHC,1,700000.0,700000.0,2021,kohlstewart
1599,Scott Heineman,OF,27.9,TEX,TEX,1,595000.0,595000.0,2021,scottheineman
1600,Albert Pujols,DH,41.2,LAA,LAD,1,570500.0,570500.0,2021,albertpujols


In [23]:
# now the data in salary_df and playerid are all cleaned. Time to merge them 

salaryFinalDF = pd.merge(salary_df, playerid, on='full_name_new', how='inner')
salaryFinalDF.head()

,name,position,age,from_team,to_tam,contract_length,total_salary,avg_salary,year,full_name_new,name_last,name_first,key_fangraphs,mlb_played_first,mlb_played_last,full_name
0,Albert Pujols,DH,31.8,STL,LAA,10,240000000.0,24000000.0,2012,albertpujols,pujols,albert,1177,2001.0,2021.0,albert pujols
1,Albert Pujols,DH,41.2,LAA,LAD,1,570500.0,570500.0,2021,albertpujols,pujols,albert,1177,2001.0,2021.0,albert pujols
2,Prince Fielder,DH,27.7,MIL,DET,9,214000000.0,23777778.0,2012,princefielder,fielder,prince,4613,2005.0,2016.0,prince fielder
3,Jose Reyes,SS,28.4,NYM,MIA,6,106000000.0,17666667.0,2012,josereyes,reyes,jose,1736,2003.0,2018.0,jose reyes
4,Jose Reyes,SS,33.0,COL,NYM,1,507500.0,507500.0,2016,josereyes,reyes,jose,1736,2003.0,2018.0,jose reyes


In [24]:
# drop unneeded columns in new DF

salaryFinalDF = salaryFinalDF.drop(columns=['full_name_new', 'name_last', 'name_first', 'full_name'])
print(salaryFinalDF.shape)
salaryFinalDF.head()

(1140, 12)


,name,position,age,from_team,to_tam,contract_length,total_salary,avg_salary,year,key_fangraphs,mlb_played_first,mlb_played_last
0,Albert Pujols,DH,31.8,STL,LAA,10,240000000.0,24000000.0,2012,1177,2001.0,2021.0
1,Albert Pujols,DH,41.2,LAA,LAD,1,570500.0,570500.0,2021,1177,2001.0,2021.0
2,Prince Fielder,DH,27.7,MIL,DET,9,214000000.0,23777778.0,2012,4613,2005.0,2016.0
3,Jose Reyes,SS,28.4,NYM,MIA,6,106000000.0,17666667.0,2012,1736,2003.0,2018.0
4,Jose Reyes,SS,33.0,COL,NYM,1,507500.0,507500.0,2016,1736,2003.0,2018.0


In [25]:
# for unknow reason, there are a few entires where the salary year info is later than the mlb_played_last, thus we need to investigate
salaryFinalDF[salaryFinalDF['year'] - salaryFinalDF['mlb_played_last'] > 1]
# hypothesis: the player signed their next FA contract from over 1 yr before FA, then retired before the next contract starts


,name,position,age,from_team,to_tam,contract_length,total_salary,avg_salary,year,key_fangraphs,mlb_played_first,mlb_played_last
336,Rich Harden,SP,31.0,OAK,MIN,1,1000000.0,1000000.0,2013,1772,2003.0,2011.0
418,Johan Santana,SP,34.9,NYM,BAL,1,8050000.0,8050000.0,2014,755,2000.0,2012.0
419,Johan Santana,SP,35.9,BAL,TOR,1,2500000.0,2500000.0,2015,755,2000.0,2012.0
425,Josh Johnson,SP,30.9,SD,SD,1,1000000.0,1000000.0,2015,4567,2005.0,2013.0
634,Pedro Feliciano,RP,38.4,STL,CHC,1,700000.0,700000.0,2015,1601,2002.0,2013.0
762,Roger Bernadina,LF,31.6,COL,NYM,1,750000.0,750000.0,2016,6421,2008.0,2014.0
763,Carlos Quentin,LF,33.4,SEA,MIN,1,750000.0,750000.0,2016,6274,2006.0,2014.0
768,Maikel Cleto,RP,27.2,CHW,ATL,1,507500.0,507500.0,2016,5529,2011.0,2014.0
838,Jacob Lindgren,RP,23.7,NYY,ATL,1,1025000.0,1025000.0,2017,16215,2015.0,2015.0
1131,Felix Hernandez,SP,34.8,ATL,BAL,1,1000000.0,1000000.0,2021,4772,2005.0,2019.0


In [26]:
# Still, we will drop the entires if there is >1 gap between salary year and mlb_played_last, since there will be missing years in production stats prior to saalary year

index_names = salaryFinalDF[salaryFinalDF['year'] - salaryFinalDF['mlb_played_last'] > 1].index

salaryFinalDF.drop(index_names, inplace = True)


In [27]:
# also we need to drop all pitchers since we're only considering batters
drop_SP = salaryFinalDF[salaryFinalDF['position'] == 'SP'].index
salaryFinalDF.drop(drop_SP, inplace = True)
drop_RP = salaryFinalDF[salaryFinalDF['position'] == 'RP'].index
salaryFinalDF.drop(drop_RP, inplace = True)

# reset index
salaryFinalDF = salaryFinalDF.reset_index(drop=True)

In [28]:
# change 'year' column to 'year-fa': year free agency

salaryFinalDF.rename(columns={'year': 'year_fa'}, inplace=True)

In [29]:
# replace all outfield positions with: OF
salaryFinalDF.position.replace(['LF', 'CF', 'RF'], 'OF', inplace=True)

In [30]:
pd.set_option('display.max_rows', 10)
salaryFinalDF.position.value_counts()

OF    177
C      95
2B     68
3B     59
1B     58
DH     36
SS     34
Name: position, dtype: int64

In [31]:
unique_ID = list(salaryFinalDF.key_fangraphs.unique())
print(f"Total number of unique ID's: {len(unique_ID)}")
unique_ID[:10]

Total number of unique ID's: 341


[1177, 4613, 1736, 1002, 1534, 589, 2103, 1572, 88, 6104]

## import mlb batting stats from 07-20 and clean data furthermore
1. Remove entries with BA less than 400:
2. Remove unwanted variables

In [32]:
# import mlb all batting stats from 2007-2020
batting_df = pd.read_csv('mlb-batting.csv')
batting_df.head()
print(batting_df.shape)

(13069, 320)


In [33]:
# drop rows with AB < 400
# batting_df = batting_df.drop(batting_df[batting_df['AB']<400].index)
# batting_df.shape

In [34]:
# drop all unwanted stats
pd.set_option('display.max_columns', None)
batting_df.head()

,Unnamed: 0,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG,GB,FB,LD,IFFB,Pitches,Balls,Strikes,IFH,BU,BUH,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,wOBA,wRAA,wRC,Bat,Fld,Rep,Pos,RAR,WAR,Dol,Spd,wRC+,WPA,-WPA,+WPA,RE24,REW,pLI,phLI,PH,WPA/LI,Clutch,FB% (Pitch),FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,BsR,FA% (sc),FT% (sc),FC% (sc),FS% (sc),FO% (sc),SI% (sc),SL% (sc),CU% (sc),KC% (sc),EP% (sc),CH% (sc),SC% (sc),KN% (sc),UN% (sc),vFA (sc),vFT (sc),vFC (sc),vFS (sc),vFO (sc),vSI (sc),vSL (sc),vCU (sc),vKC (sc),vEP (sc),vCH (sc),vSC (sc),vKN (sc),FA-X (sc),FT-X (sc),FC-X (sc),FS-X (sc),FO-X (sc),SI-X (sc),SL-X (sc),CU-X (sc),KC-X (sc),EP-X (sc),CH-X (sc),SC-X (sc),KN-X (sc),FA-Z (sc),FT-Z (sc),FC-Z (sc),FS-Z (sc),FO-Z (sc),SI-Z (sc),SL-Z (sc),CU-Z (sc),KC-Z (sc),EP-Z (sc),CH-Z (sc),SC-Z (sc),KN-Z (sc),wFA (sc),wFT (sc),wFC (sc),wFS (sc),wFO (sc),wSI (sc),wSL (sc),wCU (sc),wKC (sc),wEP (sc),wCH (sc),wSC (sc),wKN (sc),wFA/C (sc),wFT/C (sc),wFC/C (sc),wFS/C (sc),wFO/C (sc),wSI/C (sc),wSL/C (sc),wCU/C (sc),wKC/C (sc),wEP/C (sc),wCH/C (sc),wSC/C (sc),wKN/C (sc),O-Swing% (sc),Z-Swing% (sc),Swing% (sc),O-Contact% (sc),Z-Contact% (sc),Contact% (sc),Zone% (sc),Pace,Def,wSB,UBR,Age Rng,Off,Lg,wGDP,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,LD+%,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA
0,0,13611,2018,Mookie Betts,BOS,25,136,520,614,180,96,47,5,32,129,80,81,8,91,8,5,0,5,30,6,0.346,147,195,92,18,2582,1095,1487,14,0,0,0.132,0.148,0.89,0.438,0.640,1.078,0.294,0.368,0.0075,0.212,0.339,0.449,0.092,0.164,0.095,0.0,0.449,67.2,139,62.4,16.8,18.4,-5.3,101.1,10.4,$83.3,7.0,185,5.77,-8.16,13.93,62.93,6.11,0.92,3.62,4,6.44,-0.15,0.547,93.2,0.199,84.5,0.050,88.7,0.073,79.1,0.114,84.4,0.017,85.4,NaN,NaN,0.009,NaN,40.1,11.2,4.2,1.1,11.1,2.3,NaN,2.84,2.19,3.28,0.58,3.79,5.06,NaN,0.198,0.570,0.356,0.706,0.930,0.859,0.426,0.562,0.050,6.9,0.350,0.097,0.047,0.022,NaN,0.101,0.204,0.058,0.015,NaN,0.106,NaN,NaN,NaN,93.5,92.8,88.4,85.2,NaN,92.1,84.6,78.4,80.8,NaN,84.4,NaN,NaN,-2.4,-6.1,0.4,-4.1,NaN,-1.6,1.6,2.9,4.5,NaN,0.3,NaN,NaN,9.6,6.0,4.9,3.2,NaN,5.4,1.2,-5.3,-5.8,NaN,4.5,NaN,NaN,18.3,13.3,3.1,2.5,NaN,7.1,9.8,1.5,2.5,NaN,10.1,NaN,NaN,2.03,5.33,2.58,4.34,NaN,2.72,1.87,1.03,6.59,NaN,3.72,NaN,NaN,0.158,0.546,0.355,0.635,0.919,0.857,0.509,NaN,11.6,3.1,2.7,25 - 25,69.3,1.9,1.1,0.472,0.348,0.180,0.122,0.433,0.445,0.332,0.105,NaN,0.070,0.366,0.046,0.026,NaN,NaN,0.173,0.201,0.0,84.6,NaN,79.0,93.6,88.6,84.6,NaN,NaN,92.6,84.7,93.5,0.4,NaN,3.6,-1.9,0.5,-3.2,NaN,NaN,-4.2,2.1,-6.4,3.1,NaN,-7.2,8.2,4.2,1.6,NaN,NaN,4.4,-0.2,5.9,10.5,NaN,0.9,18.2,4.0,4.0,NaN,NaN,21.4,10.5,0.0,3.82,NaN,0.48,1.91,3.30,5.95,NaN,NaN,4.75,2.00,2.04,0.162,0.539,0.355,0.647,0.917,0.857,0.512,NaN,NaN,139.0,159,68,137.0,154.0,176.0,125.0,1.00,80.0,123.0,129.0,115.0,102.0,73.0,69.0,92.0,125.0,92.3,18.5,57.0,0.131,110.6,217.0,0.5,434,0.220,0.270,NaN,NaN,NaN
1,1,10155,2013,Mike Trout,LAA,21,157,589,716,190,115

In [35]:
var_list_basic = ['IDfg', 
            'Season', 
            'Name', 
            'Team', 
            'Age', 
            'G', 
            'PA', 
            'AB', 
            'R', 
            'H',
            '2B',
            '3B',
            'HR',
            'RBI',
            'SB',
            'CS',
            'BB',
            'SO',
            'GDP',
            'HBP',
            'SH',
            'SF',
            'IBB',
            'AVG',
            'OBP',
            'SLG',
            'BABIP',
            'ISO',
            'OPS',
            'wRC',
            'wRAA',
            'wOBA',
            'wRC+',
            'WPA',
            'WAR']
# not the full list
batting_basic_df = batting_df[var_list_basic]
batting_basic_df = batting_basic_df.sort_values(by=['IDfg', 'Season'], ascending=True)
batting_basic_df.reset_index(drop=True, inplace=True)
batting_basic_df

,IDfg,Season,Name,Team,Age,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,GDP,HBP,SH,SF,IBB,AVG,OBP,SLG,BABIP,ISO,OPS,wRC,wRAA,wOBA,wRC+,WPA,WAR
0,1,2007,Alfredo Amezaga,FLA,29,133,448,400,46,105,14,9,2,30,13,7,35,52,4,4,4,5,0,0.263,0.324,0.358,0.293,0.095,0.682,45,-9.9,0.305,79,-2.52,2.0
1,1,2008,Alfredo Amezaga,FLA,30,125,337,311,41,82,13,5,3,32,8,2,19,47,6,3,4,0,1,0.264,0.312,0.367,0.303,0.103,0.679,33,-7.5,0.301,77,-0.59,1.2
2,1,2009,Alfredo Amezaga,FLA,31,27,75,69,6,15,3,0,0,5,1,1,5,16,0,0,0,1,2,0.217,0.267,0.261,0.278,0.043,0.528,3,-6.2,0.228,28,-0.90,-0.2
3,1,2011,Alfredo Amezaga,- - -,33,40,87,77,6,14,1,0,0,4,0,0,7,14,1,0,2,1,0,0.182,0.247,0.195,0.219,0.013,0.442,2,-7.4,0.208,17,-0.85,-0.6
4,2,2007,Garret Anderson,LAA,35,108,450,417,67,124,31,1,16,80,1,0,27,54,8,0,0,6,9,0.297,0.336,0.492,0.306,0.194,0.827,62,6.3,0.348,110,-0.02,1.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13064,22581,2020,Alejandro Kirk,TOR,21,9,25,24,4,9,2,0,1,3,0,0,1,4,0,0,0,0,0,0.375,0.400,0.583,0.421,0.208,0.983,5,2.1,0.418,165,0.08,0.2
13065,23378,2020,Jose Barrero,CIN,22,24,68,67,4,13,0,0,0,2,1,1,1,26,1,0,0,0,0,0.194,0.206,0.194,0.317,0.000,0.400,0,-8.1,0.179,0,-0.98,-0.6
13066,24618,2020,Ryan Jeffers,MIN,23,26,62,55,5,15,0,0,3,7,0,0,5,19,0,2,0,0,0,0.273,0.355,0.436,0.364,0.164,0.791,9,1.4,0.346,120,-0.04,0.5
13067,27459,2020,Yoshi Tsutsugo,TBR,28,51,185,157,27,31,5,1,8,24,0,0,26,50,5,1,0,1,1,0.197,0.314,0.395,0.230,0.197,0.708,21,-1.7,0.309,99,0.01,0.3


### create player position tabel for later use

In [36]:
# positions_df = salaryFinalDF.drop_duplicates(subset=['key_fangraphs'])
# positions_df = positions_df[['name', 'position', 'key_fangraphs']]

# Merge salary and batting for EDA

In [37]:
# re_index batting_basic_df
batting_basic_df.reset_index(drop=True, inplace=True)

In [38]:
batting_basic_df = batting_basic_df.set_index(['IDfg'])
batting_basic_df.sort_index(level=['IDfg'], inplace=True)

In [39]:
batting_basic_df

,Season,Name,Team,Age,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,GDP,HBP,SH,SF,IBB,AVG,OBP,SLG,BABIP,ISO,OPS,wRC,wRAA,wOBA,wRC+,WPA,WAR
IDfg,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2007,Alfredo Amezaga,FLA,29,133,448,400,46,105,14,9,2,30,13,7,35,52,4,4,4,5,0,0.263,0.324,0.358,0.293,0.095,0.682,45,-9.9,0.305,79,-2.52,2.0
1,2008,Alfredo Amezaga,FLA,30,125,337,311,41,82,13,5,3,32,8,2,19,47,6,3,4,0,1,0.264,0.312,0.367,0.303,0.103,0.679,33,-7.5,0.301,77,-0.59,1.2
1,2009,Alfredo Amezaga,FLA,31,27,75,69,6,15,3,0,0,5,1,1,5,16,0,0,0,1,2,0.217,0.267,0.261,0.278,0.043,0.528,3,-6.2,0.228,28,-0.90,-0.2
1,2011,Alfredo Amezaga,- - -,33,40,87,77,6,14,1,0,0,4,0,0,7,14,1,0,2,1,0,0.182,0.247,0.195,0.219,0.013,0.442,2,-7.4,0.208,17,-0.85,-0.6
2,2007,Garret Anderson,LAA,35,108,450,417,67,124,31,1,16,80,1,0,27,54,8,0,0,6,9,0.297,0.336,0.492,0.306,0.194,0.827,62,6.3,0.348,110,-0.02,1.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22581,2020,Alejandro Kirk,TOR,21,9,25,24,4,9,2,0,1,3,0,0,1,4,0,0,0,0,0,0.375,0.400,0.583,0.421,0.208,0.983,5,2.1,0.418,165,0.08,0.2
23378,2020,Jose Barrero,CIN,22,24,68,67,4,13,0,0,0,2,1,1,1,26,1,0,0,0,0,0.194,0.206,0.194,0.317,0.000,0.400,0,-8.1,0.179,0,-0.98,-0.6
24618,2020,Ryan Jeffers,MIN,23,26,62,55,5,15,0,0,3,7,0,0,5,19,0,2,0,0,0,0.273,0.355,0.436,0.364,0.164,0.791,9,1.4,0.346,120,-0.04,0.5


## Select rows based on FA year

In [40]:
# aggregate batting_basic based on FA year
agg_method = {'Age':'max',
             'G':'sum',
             'PA':'sum',
             'AB':'sum',
             'R':'sum',
             'H':'sum',
             '2B':'sum',
             '3B':'sum',
             'HR':'sum',
             'RBI':'sum',
             'SB':'sum',
             'CS':'sum',
             'BB':'sum',
             'SO':'sum',
             'GDP':'sum',
             'HBP':'sum',
             'SH':'sum',
             'SF':'sum',
             'IBB':'sum',
             'AVG':'mean',
             'OBP':'mean',
             'SLG':'mean',
             'BABIP':'mean',
             'ISO':'mean',
             'OPS':'mean',
             'wRC':'sum',
             'wRAA':'sum',
             'wRC+':'sum',
             'WPA':'sum',
             'WAR':'sum'}
batting_aggDF = pd.DataFrame()
for row in salaryFinalDF.itertuples():
    selected_years = [row.year_fa-1, row.year_fa-2, row.year_fa-3, row.year_fa-4, row.year_fa-5,]
    player = batting_basic_df.loc[row.key_fangraphs]
    player = player[player['Season'].isin(selected_years)].groupby(by=['IDfg','Name']).agg(agg_method)
    player['Year_FA']=row.year_fa
    player['Salary']=row.avg_salary
    batting_aggDF = batting_aggDF.append(player)

In [41]:
batting_aggDF

Age    G    PA    AB    R    H   2B  3B   HR  RBI  \
IDfg  Name                                                                   
1177  Albert Pujols       31  772  3371  2823  543  914  195   3  195  571   
      Albert Pujols       40  588  2492  2294  244  570   86   0  102  402   
4613  Prince Fielder      27  802  3500  2899  487  825  161   8  200  565   
1736  Jose Reyes          28  614  2883  2616  434  776  140  59   48  238   
      Jose Reyes          32  638  2895  2652  396  785  146  34   44  242   
...                      ...  ...   ...   ...  ...  ...  ...  ..  ...  ...   
14109 Albert Almora Jr.   26  489  1316  1224  160  332   63   4   28  134   
13329 Jake Lamb           29  465  1792  1541  237  362   82  15   74  267   
5486  Abraham Almonte     31  210   591   530   76  122   32   7    8   49   
14145 Daniel Robertson    26  250   855   728   95  170   32   3   16   74   
14453 Scott Heineman      27   49   139   127   14   24    9   0    3   14   

                          SB  CS   BB   SO  GDP  HBP  SH  SF  IBB     AVG  \
IDfg  Name                                                                  
1177  Albert Pujols       48  18  482  310  118   29   0  37  153  0.3244   
      Albert Pujols       11   0  166  326   87   10   0  22   16  0.2444   
4613  Prince Fielder       9   8  505  637   64   66   0  30  110  0.2846   
1736  Jose Reyes         214  55  235  283   30    4  16  12   35  0.2950   
      Jose Reyes         148  32  204  279   31    2  18  19   21  0.2962   
...                      ...  ..  ...  ...  ...  ...  ..  ..  ...     ...   
14109 Albert Almora Jr.    4   4   67  227   33    5  10  10    6  0.2528   
13329 Jake Lamb           14   8  217  451   37   19   0  15   19  0.2210   
5486  Abraham Almonte     13   4   52  136   14    2   2   5    1  0.2114   
14145 Daniel Robertson     5   5   99  215   22   23   1   4    0  0.2535   
14453 Scott Heineman       4   2   11   31    4    1   0   0    0  0.1835   

                             OBP      SLG    BABIP      ISO     OPS  wRC  \
IDfg  Name                                                                 
1177  Albert Pujols      0.42280  0.60320  0.30600  0.27880  1.0258  675   
      Albert Pujols      0.29460  0.41580  0.24480  0.17120  0.7102  272   
4613  Prince Fielder     0.39900  0.55280  0.29860  0.26800  0.9516  629   
1736  Jose Reyes         0.35440  0.44240  0.31600  0.14760  0.7968  394   
      Jose Reyes         0.34440  0.42580  0.31580  0.12960  0.7702  371   
...                          ...      ...      ...      ...     ...  ...   
14109 Albert Almora Jr.  0.30100  0.37180  0.29660  0.11920  0.6724  141   
13329 Jake Lamb          0.32040  0.40980  0.26680  0.18880  0.7300  237   
5486  Abraham Almonte    0.30400  0.33800  0.27100  0.12660  0.6422   56   
14145 Daniel Robertson   0.35475  0.34225  0.34125  0.08825  0.6970   99   
14453 Scott Heineman     0.24550  0.32100  0.21950  0.13750  0.5665   10   

                          wRAA  wRC+    WPA   WAR  Year_FA      Salary  
IDfg  Name                                                              
1177  Albert Pujols      277.2   830  29.50  35.5     2012  24000000.0  
      Albert Pujols      -28.9   441   1.33  -2.3     2021    570500.0  
4613  Prince Fielder     216.0   735  26.24  19.2     2012  23777778.0  
1736  Jose Reyes          53.7   566   4.12  20.4     2012  17666667.0  
      Jose Reyes          49.3   553   8.78  16.6     2016    507500.0  
...                        ...   ...    ...   ...      ...         ...  
14109 Albert Almora Jr.  -18.0   391  -4.20   2.0     2021   1250000.0  
13329 Jake Lamb           21.4   457   5.56   5.0     2021   1000000.0  
5486  Abraham Almonte    -15.0   362  -2.92   0.0     2021    990000.0  
14145 Daniel Robertson    -5.9   396  -0.18   2.4     2021    900000.0  
14453 Scott Heineman      -7.4    91  -1.30  -0.5     2021    595000.0  

[527 rows x 32 columns]

## natural log salaries

In [42]:
batting_aggDF.insert(len(batting_aggDF.columns), 'Salary_log',
         np.log(batting_aggDF['Salary']))

In [43]:
batting_aggDF

Age    G    PA    AB    R    H   2B  3B   HR  RBI  \
IDfg  Name                                                                   
1177  Albert Pujols       31  772  3371  2823  543  914  195   3  195  571   
      Albert Pujols       40  588  2492  2294  244  570   86   0  102  402   
4613  Prince Fielder      27  802  3500  2899  487  825  161   8  200  565   
1736  Jose Reyes          28  614  2883  2616  434  776  140  59   48  238   
      Jose Reyes          32  638  2895  2652  396  785  146  34   44  242   
...                      ...  ...   ...   ...  ...  ...  ...  ..  ...  ...   
14109 Albert Almora Jr.   26  489  1316  1224  160  332   63   4   28  134   
13329 Jake Lamb           29  465  1792  1541  237  362   82  15   74  267   
5486  Abraham Almonte     31  210   591   530   76  122   32   7    8   49   
14145 Daniel Robertson    26  250   855   728   95  170   32   3   16   74   
14453 Scott Heineman      27   49   139   127   14   24    9   0    3   14   

                          SB  CS   BB   SO  GDP  HBP  SH  SF  IBB     AVG  \
IDfg  Name                                                                  
1177  Albert Pujols       48  18  482  310  118   29   0  37  153  0.3244   
      Albert Pujols       11   0  166  326   87   10   0  22   16  0.2444   
4613  Prince Fielder       9   8  505  637   64   66   0  30  110  0.2846   
1736  Jose Reyes         214  55  235  283   30    4  16  12   35  0.2950   
      Jose Reyes         148  32  204  279   31    2  18  19   21  0.2962   
...                      ...  ..  ...  ...  ...  ...  ..  ..  ...     ...   
14109 Albert Almora Jr.    4   4   67  227   33    5  10  10    6  0.2528   
13329 Jake Lamb           14   8  217  451   37   19   0  15   19  0.2210   
5486  Abraham Almonte     13   4   52  136   14    2   2   5    1  0.2114   
14145 Daniel Robertson     5   5   99  215   22   23   1   4    0  0.2535   
14453 Scott Heineman       4   2   11   31    4    1   0   0    0  0.1835   

                             OBP      SLG    BABIP      ISO     OPS  wRC  \
IDfg  Name                                                                 
1177  Albert Pujols      0.42280  0.60320  0.30600  0.27880  1.0258  675   
      Albert Pujols      0.29460  0.41580  0.24480  0.17120  0.7102  272   
4613  Prince Fielder     0.39900  0.55280  0.29860  0.26800  0.9516  629   
1736  Jose Reyes         0.35440  0.44240  0.31600  0.14760  0.7968  394   
      Jose Reyes         0.34440  0.42580  0.31580  0.12960  0.7702  371   
...                          ...      ...      ...      ...     ...  ...   
14109 Albert Almora Jr.  0.30100  0.37180  0.29660  0.11920  0.6724  141   
13329 Jake Lamb          0.32040  0.40980  0.26680  0.18880  0.7300  237   
5486  Abraham Almonte    0.30400  0.33800  0.27100  0.12660  0.6422   56   
14145 Daniel Robertson   0.35475  0.34225  0.34125  0.08825  0.6970   99   
14453 Scott Heineman     0.24550  0.32100  0.21950  0.13750  0.5665   10   

                          wRAA  wRC+    WPA   WAR  Year_FA      Salary  \
IDfg  Name                                                               
1177  Albert Pujols      277.2   830  29.50  35.5     2012  24000000.0   
      Albert Pujols      -28.9   441   1.33  -2.3     2021    570500.0   
4613  Prince Fielder     216.0   735  26.24  19.2     2012  23777778.0   
1736  Jose Reyes          53.7   566   4.12  20.4     2012  17666667.0   
      Jose Reyes          49.3   553   8.78  16.6     2016    507500.0   
...                        ...   ...    ...   ...      ...         ...   
14109 Albert Almora Jr.  -18.0   391  -4.20   2.0     2021   1250000.0   
13329 Jake Lamb           21.4   457   5.56   5.0     2021   1000000.0   
5486  Abraham Almonte    -15.0   362  -2.92   0.0     2021    990000.0   
14145 Daniel Robertson    -5.9   396  -0.18   2.4     2021    900000.0   
14453 Scott Heineman      -7.4    91  -1.30  -0.5     2021    595000.0   

                         Salary_log  
IDfg  Name        